In [ ]:
def ReadFile(status):
	global witchFile

	if status == 1:
		data = pd.read_csv('Data/train.csv', sep=',', low_memory=0, usecols=[1, 2, 3, 4, 5, 7], nrows=100)

		newData = pd.DataFrame(data)
		newData['newColumn'] = 0
		newData.to_csv('Data/train1.csv')
						
		witchFile = 'Data/train1'

		return newData,newData.groupby(['Log_Date', 'FROM', 'TO']).size()
	else:
		data = pd.read_csv('Data/test.csv', sep=',', low_memory=0, usecols=[1, 2, 3, 4, 5, 7], nrows=100)

		newData = pd.DataFrame(data)
		newData['newColumn'] = 0
		newData.to_csv('Data/test1.csv')

		witchFile = 'Data/test1.csv'

		return newData,newData.groupby(['Log_Date', 'FROM', 'TO']).size()

This method is for read csv file 
(status) is to know witch file we want to read (train / test)
after that we add new column to csv to add -- in feature

In [ ]:
def WriteToCSV(csv,row,column,value):
	csv.at[row, column] = value
	csv.to_csv(witchFile + '.csv', index=False)

this method use to write value that we want (in this project --) to new csv that create in past code block

In [ ]:
def preprocess(train_data,train_label_values):
    global temp
    
    train_label_titles = train_label_values.index.values
    
    dic = {}
    for j in range(0, len(train_label_values)):
        dic.__setitem__(train_label_titles[j], train_label_values[j])    

    train_data = deepcopy(train_data.values)
    train_label_values = deepcopy(train_label_values.values)

    i = 0
    counter = []    
    train_label = []
    temp = np.resize(train_data, (6000, 9))

    for td in train_data:
        label = (td[0], td[2], td[3])
        train_label.append(dic[label])
        date = jdatetime.datetime.strptime(td[0], "%Y/%m/%d")

        d = date.day
        m = date.strftime("%m")
        w = date.weekday()
        s = (int(m) % 12 + 3) // 3

        temp[i][0] = d
        temp[i][1] = int(m)
        temp[i][2] = w
        temp[i][3] = s
        temp[i][4] = td[1]
        temp[i][5] = td[2]
        temp[i][6] = td[3]
        temp[i][7] = td[4]
        temp[i][8] = td[5]

        i += 1

    # train_label = keras.utils.to_categorical(train_label_values, 1)
    # test_label = keras.utils.to_categorical(test_label, 10)
    train_label = np.array(train_label)
    train_label = train_label.astype('float32')

    if i!=100:
        temp = temp.astype('float32')

    return temp, train_label